In [11]:
  !pip install wandb
import wandb
!wandb login 58a0b576fd5221cd0d63b154deaabbe535e853c6

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


In [2]:
# =======================
# Imports and Sweep Config
# =======================
# =======================
# Imports and Sweep Config
# =======================
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import wandb
import os
import math

sweep_config = {
    'method': 'bayes',
    'metric': {
        'name': 'val_loss',
        'goal': 'minimize'
    },
    'parameters': {
        'embedding_dim': {'values': [32,64,128,256]},
        'hidden_dim': {'values': [32,64,128,256]},
        'enc_layers': {'values': [1,2,3]},
        'dec_layers': {'values': [1,2,3]},
        'cell_type': {'values': ['GRU', 'LSTM', 'RNN']},
        'dropout': {'values': [0.2,0.3,0.5]},
        'epochs': {'values': [10,15]},
        'beam_size': {'values': [1,3,5]},
        'attention_type': {'values': ['dot', 'general', 'concat']},
        'batch_size': {'values': [64,128,256]},
        'learning_rate': {'values': [0.001,0.0005,0.0001]}
    }
}

# =======================
# Default Config
# =======================
default_config = {
    'embedding_dim': 32,
    'hidden_dim': 64,
    'enc_layers': 1,
    'dec_layers': 1,
    'cell_type': 'LSTM',
    'dropout': 0.2,
    'epochs': 10,
    'beam_size': 1,
    'attention_type': 'general',
    'batch_size': 64,
    'learning_rate': 0.001
}


# =======================
# Vocabulary
# =======================
class Vocab:
    def __init__(self):
        self.char2idx = {"<pad>": 0, "<sos>": 1, "<eos>": 2}
        self.idx2char = {0: "<pad>", 1: "<sos>", 2: "<eos>"}
        self.size = 3

    def build(self, texts):
        for text in texts:
            for char in text:
                if char not in self.char2idx:
                    self.char2idx[char] = self.size
                    self.idx2char[self.size] = char
                    self.size += 1

    def encode(self, text):
        return [self.char2idx[c] for c in text]

    def decode(self, idxs):
        return ''.join([self.idx2char[i] for i in idxs if i > 2])

# =======================
# Dataset
# =======================
class TransliterationDataset(Dataset):
    def __init__(self, filepath, inp_vocab, out_vocab):
        self.pairs = []
        with open(filepath, encoding='utf-8') as f:
            for line in f:
                fields = line.strip().split('\t')
                if len(fields) < 2:
                    continue
                lat, dev = fields[0], fields[1]
                self.pairs.append((lat, dev))
        inp_vocab.build([p[0] for p in self.pairs])
        out_vocab.build([p[1] for p in self.pairs])
        self.inp_vocab = inp_vocab
        self.out_vocab = out_vocab

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        lat, dev = self.pairs[idx]
        x = self.inp_vocab.encode(lat)
        y = [self.out_vocab.char2idx["<sos>"]] + self.out_vocab.encode(dev) + [self.out_vocab.char2idx["<eos>"]]
        return torch.tensor(x), torch.tensor(y)

def collate_fn(batch):
    x_batch, y_batch = zip(*batch)
    x_lens = [len(x) for x in x_batch]
    y_lens = [len(y) for y in y_batch]
    x_pad = nn.utils.rnn.pad_sequence(x_batch, batch_first=True, padding_value=0)
    y_pad = nn.utils.rnn.pad_sequence(y_batch, batch_first=True, padding_value=0)
    return x_pad, y_pad, torch.tensor(x_lens), torch.tensor(y_lens)

# =======================
# Attention Mechanism
# =======================
class Attention(nn.Module):
    def __init__(self, hidden_dim, attention_type='general'):
        super().__init__()
        self.attention_type = attention_type
        
        if attention_type == 'general':
            self.attn = nn.Linear(hidden_dim, hidden_dim)
        elif attention_type == 'concat':
            self.attn = nn.Linear(hidden_dim * 2, hidden_dim)
            self.v = nn.Linear(hidden_dim, 1, bias=False)
            
    def forward(self, hidden, encoder_outputs, mask=None):
        # hidden: [batch_size, hidden_dim]
        # encoder_outputs: [batch_size, src_len, hidden_dim]
        
        batch_size, src_len, hidden_dim = encoder_outputs.shape
        
        # For dot and general attention
        if self.attention_type == 'dot':
            # Calculate dot product between hidden and encoder_outputs
            energy = torch.bmm(encoder_outputs, hidden.unsqueeze(2)).squeeze(2)
            # energy: [batch_size, src_len]
            
        elif self.attention_type == 'general':
            # Calculate general attention
            energy = torch.bmm(encoder_outputs, self.attn(hidden).unsqueeze(2)).squeeze(2)
            # energy: [batch_size, src_len]
            
        elif self.attention_type == 'concat':
            # Repeat hidden across source length
            hidden_expanded = hidden.unsqueeze(1).repeat(1, src_len, 1)
            # Concatenate hidden and encoder_outputs
            energy = self.v(torch.tanh(self.attn(torch.cat((hidden_expanded, encoder_outputs), dim=2)))).squeeze(2)
            # energy: [batch_size, src_len]
        
        # Apply mask if provided (for padding)
        if mask is not None:
            energy = energy.masked_fill(mask == 0, -1e10)
        
        # Apply softmax to get attention weights
        attention_weights = F.softmax(energy, dim=1)
        # attention_weights: [batch_size, src_len]
        
        # Apply attention weights to encoder outputs
        context = torch.bmm(attention_weights.unsqueeze(1), encoder_outputs).squeeze(1)
        # context: [batch_size, hidden_dim]
        
        return context, attention_weights

# =======================
# Encoder and Decoder
# =======================
class Encoder(nn.Module):
    def __init__(self, vocab_size, emb_dim, hidden_dim, num_layers, cell_type, dropout):
        super().__init__()
        self.cell_type = cell_type
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        self.embedding = nn.Embedding(vocab_size, emb_dim, padding_idx=0)
        rnn_class = {"GRU": nn.GRU, "LSTM": nn.LSTM, "RNN": nn.RNN}[cell_type]
        self.rnn = rnn_class(emb_dim, hidden_dim, num_layers, batch_first=True, bidirectional=False, dropout=dropout if num_layers > 1 else 0)

    def forward(self, x, lengths):
        embedded = self.embedding(x)
        packed = nn.utils.rnn.pack_padded_sequence(embedded, lengths.cpu(), batch_first=True, enforce_sorted=False)
        outputs, hidden = self.rnn(packed)
        outputs, _ = nn.utils.rnn.pad_packed_sequence(outputs, batch_first=True)
        
        return outputs, hidden

class AttentionDecoder(nn.Module):
    def __init__(self, vocab_size, emb_dim, hidden_dim, num_layers, cell_type, dropout, attention_type):
        super().__init__()
        self.cell_type = cell_type
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        self.embedding = nn.Embedding(vocab_size, emb_dim, padding_idx=0)
        rnn_class = {"GRU": nn.GRU, "LSTM": nn.LSTM, "RNN": nn.RNN}[cell_type]
        
        # Input to RNN will be embedding + context vector
        self.rnn = rnn_class(emb_dim + hidden_dim, hidden_dim, num_layers, batch_first=True, dropout=dropout if num_layers > 1 else 0)
        
        self.attention = Attention(hidden_dim, attention_type)
        
        # Output layer combines hidden state and context vector
        self.out = nn.Linear(hidden_dim * 2, vocab_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input_token, hidden, encoder_outputs, mask=None):
        # input_token: [batch_size]
        # hidden: tuple of [num_layers, batch_size, hidden_dim] for LSTM or [num_layers, batch_size, hidden_dim] for GRU/RNN
        # encoder_outputs: [batch_size, src_len, hidden_dim]
        
        # Get the last layer's hidden state for attention
        if self.cell_type == "LSTM":
            attn_hidden = hidden[0][-1]  # Last layer's hidden state
        else:
            attn_hidden = hidden[-1]  # Last layer's hidden state
        
        # Calculate attention
        context, attention_weights = self.attention(attn_hidden, encoder_outputs, mask)
        
        # Embed input token
        embedded = self.embedding(input_token)  # [batch_size, emb_dim]
        
        # Concatenate embedding and context vector
        rnn_input = torch.cat((embedded, context), dim=1).unsqueeze(1)  # [batch_size, 1, emb_dim + hidden_dim]
        
        # Pass through RNN
        output, hidden = self.rnn(rnn_input, hidden)
        
        # Get the output from the last layer
        if self.cell_type == "LSTM":
            output_hidden = hidden[0][-1]  # Last layer's hidden state
        else:
            output_hidden = hidden[-1]  # Last layer's hidden state
        
        # Concatenate output and context for prediction
        output = torch.cat((output_hidden, context), dim=1)
        
        # Apply dropout and predict
        output = self.dropout(output)
        prediction = self.out(output)
        
        return prediction, hidden, attention_weights

# =======================
# Seq2Seq Model with Beam Search
# =======================
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, enc_layers, dec_layers, cell_type, device, beam_size=1):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.cell_type = cell_type
        self.enc_layers = enc_layers
        self.dec_layers = dec_layers
        self.device = device
        self.beam_size = beam_size

    def create_mask(self, src, src_lens):
        # Create mask for attention (1 for valid positions, 0 for padding)
        batch_size = src.size(0)
        max_len = src.size(1)
        mask = torch.zeros(batch_size, max_len, device=self.device)
        for i, length in enumerate(src_lens):
            mask[i, :length] = 1
        return mask

    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        src_data, src_lens = src
        batch_size, trg_len = trg.size()
        vocab_size = self.decoder.out.out_features
        
        # Store outputs, attention weights
        outputs = torch.zeros(batch_size, trg_len, vocab_size).to(self.device)
        attentions = torch.zeros(batch_size, trg_len, src_data.size(1)).to(self.device)
        
        # Encode
        encoder_outputs, enc_hidden = self.encoder(src_data, src_lens)
        
        # Create mask for attention
        mask = self.create_mask(src_data, src_lens)
        
        # Match encoder and decoder layers
        if self.cell_type == "LSTM":
            h, c = enc_hidden
            h = self._match_layers(h)
            c = self._match_layers(c)
            dec_hidden = (h, c)
        else:
            dec_hidden = self._match_layers(enc_hidden)
        
        # First input to the decoder is the <sos> token
        input_token = trg[:, 0]
        
        for t in range(1, trg_len):
            output, dec_hidden, attn_weights = self.decoder(
                input_token, dec_hidden, encoder_outputs, mask
            )
            
            outputs[:, t] = output
            attentions[:, t] = attn_weights
            
            # Teacher forcing
            teacher_force = torch.rand(1).item() < teacher_forcing_ratio
            top1 = output.argmax(1)
            input_token = trg[:, t] if teacher_force else top1
            
        return outputs, attentions

    def _match_layers(self, hidden):
        # Match encoder and decoder layers
        if self.enc_layers == self.dec_layers:
            return hidden
        elif self.enc_layers > self.dec_layers:
            return hidden[:self.dec_layers]
        else:
            pad = hidden.new_zeros((self.dec_layers - self.enc_layers, *hidden.shape[1:]))
            return torch.cat([hidden, pad], dim=0)
    
    def beam_search(self, src, max_len=50, sos_idx=1, eos_idx=2):
        src_data, src_lens = src
        batch_size = src_data.size(0)
        assert batch_size == 1, "Beam search only supports batch size of 1 for now"

        # Encode
        encoder_outputs, enc_hidden = self.encoder(src_data, src_lens)
        mask = self.create_mask(src_data, src_lens)

        # Prepare initial hidden state
        if self.cell_type == "LSTM":
            h, c = enc_hidden
            h = self._match_layers(h)
            c = self._match_layers(c)
            dec_hidden = (h, c)
        else:
            dec_hidden = self._match_layers(enc_hidden)

        # Initialize beams with [score, sequence, hidden_state]
        beams = [{
            "score": 0.0,
            "seq": [sos_idx],
            "hidden": dec_hidden
        }]

        for _ in range(max_len):
            new_beams = []
            for beam in beams:
                seq = beam["seq"]
                if seq[-1] == eos_idx:
                    new_beams.append(beam)
                    continue

                input_token = torch.tensor([seq[-1]], device=self.device)
                dec_hidden = beam["hidden"]
                output, new_hidden, _ = self.decoder(input_token, dec_hidden, encoder_outputs, mask)

                log_probs = F.log_softmax(output, dim=1).squeeze(0)
                topk_log_probs, topk_indices = torch.topk(log_probs, self.beam_size)

                for log_prob, idx in zip(topk_log_probs, topk_indices):
                    new_beams.append({
                        "score": beam["score"] + log_prob.item(),
                        "seq": beam["seq"] + [idx.item()],
                        "hidden": new_hidden
                    })

            # Keep top `beam_size` beams
            beams = sorted(new_beams, key=lambda x: x["score"], reverse=True)[:self.beam_size]

            # Early stopping if all beams end with <eos>
            if all(beam["seq"][-1] == eos_idx for beam in beams):
                break

        # Choose best beam
        best_beam = max(beams, key=lambda x: x["score"])
        return best_beam["seq"]





# =======================
# Train & Eval
# =======================
def train(model, loader, criterion, optimizer, device):
    model.train()
    total_loss, total_correct, total_count = 0, 0, 0
    for src, trg, src_lens, _ in loader:
        src, trg = src.to(device), trg.to(device)
        optimizer.zero_grad()
        output, _ = model((src, src_lens), trg)
        output_dim = output.shape[-1]
        loss = criterion(output[:, 1:].reshape(-1, output_dim), trg[:, 1:].reshape(-1))
        pred = output.argmax(2)
        correct = ((pred[:, 1:] == trg[:, 1:]) & (trg[:, 1:] != 0)).sum().item()
        total_correct += correct
        total_count += (trg[:, 1:] != 0).sum().item()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    acc = 100.0 * total_correct / total_count
    print(f"Train Loss: {total_loss / len(loader):.4f}, Acc: {acc:.2f}%")
    return total_loss / len(loader), acc

def evaluate(model, loader, criterion, device):
    model.eval()
    total_loss, total_correct, total_count = 0, 0, 0
    with torch.no_grad():
        for src, trg, src_lens, _ in loader:
            src, trg = src.to(device), trg.to(device)
            output, _ = model((src, src_lens), trg, teacher_forcing_ratio=0)
            output_dim = output.shape[-1]
            loss = criterion(output[:, 1:].reshape(-1, output_dim), trg[:, 1:].reshape(-1))
            pred = output.argmax(2)
            correct = ((pred[:, 1:] == trg[:, 1:]) & (trg[:, 1:] != 0)).sum().item()
            total_correct += correct
            total_count += (trg[:, 1:] != 0).sum().item()
            total_loss += loss.item()
    acc = 100.0 * total_correct / total_count
    print(f"Val Loss: {total_loss / len(loader):.4f}, Acc: {acc:.2f}%")
    return total_loss / len(loader), acc

# =======================
# Main
# =======================
def main():
    wandb.init(config=default_config, project="dakshina-transliteration")
    config = wandb.config
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    inp_vocab, out_vocab = Vocab(), Vocab()
    train_path = "/kaggle/input/devnagiridata/hi.translit.sampled.train.tsv"
    dev_path = "/kaggle/input/devnagiridata/hi.translit.sampled.dev.tsv"
    
    train_data = TransliterationDataset(train_path, inp_vocab, out_vocab)
    dev_data = TransliterationDataset(dev_path, inp_vocab, out_vocab)
    
    # Use config.batch_size for DataLoader
    train_loader = DataLoader(train_data, batch_size=config.batch_size, 
                             shuffle=True, collate_fn=collate_fn)
    dev_loader = DataLoader(dev_data, batch_size=config.batch_size,
                           shuffle=False, collate_fn=collate_fn)

    encoder = Encoder(inp_vocab.size, config.embedding_dim, config.hidden_dim,
                     config.enc_layers, config.cell_type, config.dropout)
    decoder = AttentionDecoder(out_vocab.size, config.embedding_dim, config.hidden_dim,
                              config.dec_layers, config.cell_type, config.dropout,
                              config.attention_type)
    
    model = Seq2Seq(encoder, decoder, config.enc_layers, config.dec_layers,
                   config.cell_type, device, beam_size=config.beam_size).to(device)
    
    # Use config.learning_rate for optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=config.learning_rate)
    criterion = nn.CrossEntropyLoss(ignore_index=0)

    for epoch in range(config.epochs):
        print(f"Epoch {epoch+1}/{config.epochs}")
        train_loss, train_acc = train(model, train_loader, criterion, optimizer, device)
        val_loss, val_acc = evaluate(model, dev_loader, criterion, device)
        wandb.log({
            "train_loss": train_loss, 
            "train_acc": train_acc, 
            "val_loss": val_loss, 
            "val_acc": val_acc, 
            "epoch": epoch+1
        })

# =======================
if __name__ == '__main__':
    sweep_id = wandb.sweep(sweep_config, project="dakshina-transliteration-attention")
    wandb.agent(sweep_id, function=main, count=30)


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Create sweep with ID: 64q660zv
Sweep URL: https://wandb.ai/manglesh_dl_ass3/dakshina-transliteration-attention/sweeps/64q660zv


wandb: Agent Starting Run: kk1paf4n with config:
wandb: 	attention_type: general
wandb: 	batch_size: 256
wandb: 	beam_size: 1
wandb: 	cell_type: GRU
wandb: 	dec_layers: 1
wandb: 	dropout: 0.5
wandb: 	embedding_dim: 128
wandb: 	enc_layers: 1
wandb: 	epochs: 10
wandb: 	hidden_dim: 32
wandb: 	learning_rate: 0.001
wandb: Currently logged in as: manglesh_dlass3 (manglesh_dl_ass3) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch 1/10
Train Loss: 2.5110, Acc: 29.00%
Val Loss: 1.9818, Acc: 41.74%
Epoch 2/10
Train Loss: 1.7008, Acc: 50.11%
Val Loss: 1.3793, Acc: 56.66%
Epoch 3/10
Train Loss: 1.3229, Acc: 61.12%
Val Loss: 1.1319, Acc: 65.53%
Epoch 4/10
Train Loss: 1.1588, Acc: 66.26%
Val Loss: 1.0422, Acc: 68.36%
Epoch 5/10
Train Loss: 1.0605, Acc: 69.47%
Val Loss: 1.0014, Acc: 70.01%
Epoch 6/10
Train Loss: 1.0092, Acc: 71.01%
Val Loss: 0.9782, Acc: 70.04%
Epoch 7/10
Train Loss: 0.9667, Acc: 72.27%
Val Loss: 0.9751, Acc: 70.81%
Epoch 8/10
Train Loss: 0.9433, Acc: 72.94%
Val Loss: 0.9551, Acc: 71.02%
Epoch 9/10
Train Loss: 0.9219, Acc: 73.60%
Val Loss: 0.9505, Acc: 71.29%
Epoch 10/10
Train Loss: 0.9076, Acc: 74.02%
Val Loss: 0.9340, Acc: 71.47%


epoch,▁▂▃▃▄▅▆▆▇█
train_acc,▁▄▆▇▇█████
train_loss,█▄▃▂▂▁▁▁▁▁
val_acc,▁▅▇▇██████
val_loss,█▄▂▂▁▁▁▁▁▁
epoch,10
train_acc,74.02213
train_loss,0.90762
val_acc,71.46784
val_loss,0.93404


...

Epoch 1/15
Train Loss: 1.1083, Acc: 67.11%
Val Loss: 0.9629, Acc: 70.57%
Epoch 2/15
Train Loss: 0.6854, Acc: 79.01%
Val Loss: 0.8795, Acc: 72.39%
Epoch 3/15
Train Loss: 0.6285, Acc: 80.40%
Val Loss: 0.8690, Acc: 73.82%
Epoch 4/15
Train Loss: 0.5906, Acc: 81.47%
Val Loss: 0.8232, Acc: 74.25%
Epoch 5/15
Train Loss: 0.5742, Acc: 81.76%
Val Loss: 0.8182, Acc: 73.68%
Epoch 6/15
Train Loss: 0.5536, Acc: 82.38%
Val Loss: 0.8186, Acc: 74.91%
Epoch 7/15
Train Loss: 0.5413, Acc: 82.65%
Val Loss: 0.8081, Acc: 74.86%
Epoch 8/15
Train Loss: 0.5237, Acc: 83.29%
Val Loss: 0.8119, Acc: 74.97%
Epoch 9/15
Train Loss: 0.5111, Acc: 83.60%
Val Loss: 0.7972, Acc: 75.36%
Epoch 10/15
Train Loss: 0.5088, Acc: 83.53%
Val Loss: 0.7974, Acc: 75.48%
Epoch 11/15
Train Loss: 0.5022, Acc: 83.76%
Val Loss: 0.8097, Acc: 75.19%
Epoch 12/15
Train Loss: 0.4964, Acc: 83.88%
Val Loss: 0.8004, Acc: 75.36%
Epoch 13/15
Train Loss: 0.4877, Acc: 84.08%
Val Loss: 0.8045, Acc: 75.42%
Epoch 14/15
Train Loss: 0.4861, Acc: 84.06%
Val

epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
train_acc,▁▆▆▇▇▇▇████████
train_loss,█▃▃▂▂▂▂▂▁▁▁▁▁▁▁
val_acc,▁▄▆▆▅▇▇▇█████▇█
val_loss,█▅▄▂▂▂▂▂▁▁▂▁▁▂▁
epoch,15
train_acc,84.4082
train_loss,0.47749
val_acc,75.47814
val_loss,0.79498


wandb: Agent Starting Run: wbrsydns with config:
wandb: 	attention_type: concat
wandb: 	batch_size: 256
wandb: 	beam_size: 5
wandb: 	cell_type: GRU
wandb: 	dec_layers: 1
wandb: 	dropout: 0.5
wandb: 	embedding_dim: 256
wandb: 	enc_layers: 3
wandb: 	epochs: 10
wandb: 	hidden_dim: 256
wandb: 	learning_rate: 0.001


Epoch 1/10
Train Loss: 1.5142, Acc: 54.02%
Val Loss: 1.0001, Acc: 68.74%
Epoch 2/10
Train Loss: 0.8397, Acc: 74.15%
Val Loss: 0.8635, Acc: 72.71%
Epoch 3/10
Train Loss: 0.7281, Acc: 77.54%
Val Loss: 0.8369, Acc: 73.72%
Epoch 4/10
Train Loss: 0.6878, Acc: 78.64%
Val Loss: 0.8221, Acc: 74.04%
Epoch 5/10
Train Loss: 0.6428, Acc: 80.01%
Val Loss: 0.8310, Acc: 74.59%
Epoch 6/10
Train Loss: 0.6204, Acc: 80.64%
Val Loss: 0.8199, Acc: 74.61%
Epoch 7/10
Train Loss: 0.6094, Acc: 80.95%
Val Loss: 0.8108, Acc: 74.48%
Epoch 8/10
Train Loss: 0.5889, Acc: 81.50%
Val Loss: 0.8158, Acc: 75.18%
Epoch 9/10
Train Loss: 0.5622, Acc: 82.34%
Val Loss: 0.8262, Acc: 75.21%
Epoch 10/10
Train Loss: 0.5605, Acc: 82.30%
Val Loss: 0.8074, Acc: 75.14%


epoch,▁▂▃▃▄▅▆▆▇█
train_acc,▁▆▇▇▇█████
train_loss,█▃▂▂▂▁▁▁▁▁
val_acc,▁▅▆▇▇▇▇███
val_loss,█▃▂▂▂▁▁▁▂▁
epoch,10
train_acc,82.30252
train_loss,0.56055
val_acc,75.13961
val_loss,0.80738


In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import os
import random
import wandb

# =======================
# Best Configuration
# =======================
best_config = {
    'embedding_dim': 256,
    'hidden_dim': 256,
    'enc_layers': 2,
    'dec_layers': 2,
    'cell_type': 'LSTM',
    'dropout': 0.5,
    'epochs': 15,
    'beam_size': 5,
    'attention_type': 'concat',
    'batch_size': 256,
    'learning_rate': 0.001
}

# =======================
# Vocabulary
# =======================
class Vocab:
    def __init__(self):
        self.char2idx = {"<pad>": 0, "<sos>": 1, "<eos>": 2}
        self.idx2char = {0: "<pad>", 1: "<sos>", 2: "<eos>"}
        self.size = 3

    def build(self, texts):
        for text in texts:
            for char in text:
                if char not in self.char2idx:
                    self.char2idx[char] = self.size
                    self.idx2char[self.size] = char
                    self.size += 1

    def encode(self, text):
        return [self.char2idx[c] for c in text]

    def decode(self, idxs):
        return ''.join([self.idx2char[i] for i in idxs if i not in [0, 1, 2]])

# =======================
# Dataset
# =======================
class TransliterationDataset(Dataset):
    def __init__(self, filepath, inp_vocab, out_vocab, is_test=False):
        self.pairs = []
        with open(filepath, encoding='utf-8') as f:
            for line in f:
                fields = line.strip().split('\t')
                if len(fields) < 2:
                    continue
                lat, dev = fields[0], fields[1]
                self.pairs.append((lat, dev))
        if not is_test:
            inp_vocab.build([p[0] for p in self.pairs])
            out_vocab.build([p[1] for p in self.pairs])
        self.inp_vocab = inp_vocab
        self.out_vocab = out_vocab
        self.is_test = is_test

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        lat, dev = self.pairs[idx]
        x = self.inp_vocab.encode(lat)
        if self.is_test:
            return torch.tensor(x), lat, dev
        y = [self.out_vocab.char2idx["<sos>"]] + self.out_vocab.encode(dev) + [self.out_vocab.char2idx["<eos>"]]
        return torch.tensor(x), torch.tensor(y), lat, dev

def collate_fn(batch):
    if len(batch[0]) == 3:  # Test batch
        x_batch, lat, dev = zip(*batch)
        x_lens = [len(x) for x in x_batch]
        x_pad = nn.utils.rnn.pad_sequence(x_batch, batch_first=True, padding_value=0)
        return x_pad, lat, dev, torch.tensor(x_lens)
    else:  # Train/val batch
        x_batch, y_batch, lat, dev = zip(*batch)
        x_lens = [len(x) for x in x_batch]
        y_lens = [len(y) for y in y_batch]
        x_pad = nn.utils.rnn.pad_sequence(x_batch, batch_first=True, padding_value=0)
        y_pad = nn.utils.rnn.pad_sequence(y_batch, batch_first=True, padding_value=0)
        return x_pad, y_pad, torch.tensor(x_lens), torch.tensor(y_lens), lat, dev

# =======================
# Model Components
# =======================
class Encoder(nn.Module):
    def __init__(self, vocab_size, emb_dim, hidden_dim, num_layers, cell_type, dropout):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, emb_dim, padding_idx=0)
        rnn_class = {"GRU": nn.GRU, "LSTM": nn.LSTM, "RNN": nn.RNN}[cell_type]
        self.rnn = rnn_class(emb_dim, hidden_dim, num_layers, batch_first=True, dropout=dropout if num_layers > 1 else 0)

    def forward(self, x, lengths):
        embedded = self.embedding(x)
        packed = nn.utils.rnn.pack_padded_sequence(embedded, lengths.cpu(), batch_first=True, enforce_sorted=False)
        outputs, hidden = self.rnn(packed)
        outputs, _ = nn.utils.rnn.pad_packed_sequence(outputs, batch_first=True)
        return outputs, hidden

class Attention(nn.Module):
    def __init__(self, hidden_dim, attention_type='general'):
        super().__init__()
        self.attention_type = attention_type
        if attention_type == 'general':
            self.attn = nn.Linear(hidden_dim, hidden_dim)
        elif attention_type == 'concat':
            self.attn = nn.Linear(hidden_dim * 2, hidden_dim)
            self.v = nn.Linear(hidden_dim, 1, bias=False)

    def forward(self, hidden, encoder_outputs, mask=None):
        batch_size, src_len, hidden_dim = encoder_outputs.size()
        
        if self.attention_type == 'general':
            energy = torch.bmm(encoder_outputs, self.attn(hidden).unsqueeze(2)).squeeze(2)
        elif self.attention_type == 'concat':
            hidden_expanded = hidden.unsqueeze(1).repeat(1, src_len, 1)
            concat = torch.cat((hidden_expanded, encoder_outputs), dim=2)
            energy = self.v(torch.tanh(self.attn(concat))).squeeze(2)
        else:  # dot
            energy = torch.bmm(encoder_outputs, hidden.unsqueeze(2)).squeeze(2)
        
        if mask is not None:
            energy = energy.masked_fill(mask == 0, -1e10)
        attention_weights = F.softmax(energy, dim=1)
        context = torch.bmm(attention_weights.unsqueeze(1), encoder_outputs).squeeze(1)
        return context, attention_weights

class Decoder(nn.Module):
    def __init__(self, vocab_size, emb_dim, hidden_dim, num_layers, cell_type, dropout, attention_type):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, emb_dim, padding_idx=0)
        rnn_class = {"GRU": nn.GRU, "LSTM": nn.LSTM, "RNN": nn.RNN}[cell_type]
        self.rnn = rnn_class(emb_dim + hidden_dim, hidden_dim, num_layers, batch_first=True, dropout=dropout if num_layers > 1 else 0)
        self.attention = Attention(hidden_dim, attention_type)
        self.out = nn.Linear(hidden_dim * 2, vocab_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input_token, hidden, encoder_outputs, mask=None):
        if isinstance(hidden, tuple):  # LSTM
            attn_hidden = hidden[0][-1]
        else:  # GRU/RNN
            attn_hidden = hidden[-1]
        
        context, _ = self.attention(attn_hidden, encoder_outputs, mask)
        embedded = self.embedding(input_token)
        rnn_input = torch.cat((embedded, context), dim=1).unsqueeze(1)
        output, hidden = self.rnn(rnn_input, hidden)
        
        if isinstance(hidden, tuple):
            output_hidden = hidden[0][-1]
        else:
            output_hidden = hidden[-1]
        
        output = torch.cat((output_hidden, context), dim=1)
        output = self.dropout(output)
        prediction = self.out(output)
        return prediction, hidden, None

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def create_mask(self, src_lens, max_len):
        batch_size = len(src_lens)
        mask = torch.zeros(batch_size, max_len, device=self.device)
        for i, length in enumerate(src_lens):
            mask[i, :length] = 1
        return mask

    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        src_data, src_lens = src
        encoder_outputs, enc_hidden = self.encoder(src_data, src_lens)
        batch_size, trg_len = trg.size()
        vocab_size = self.decoder.out.out_features
        outputs = torch.zeros(batch_size, trg_len, vocab_size).to(self.device)
        
        src_len = encoder_outputs.size(1)
        mask = self.create_mask(src_lens, src_len)

        if isinstance(enc_hidden, tuple):
            dec_hidden = enc_hidden
        else:
            dec_hidden = enc_hidden

        input_token = trg[:, 0]
        for t in range(1, trg_len):
            output, dec_hidden, _ = self.decoder(input_token, dec_hidden, encoder_outputs, mask)
            outputs[:, t] = output
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.argmax(1)
            input_token = trg[:, t] if teacher_force else top1
            
        return outputs

    def predict(self, src, src_lens, max_len=30):
        self.eval()
        with torch.no_grad():
            encoder_outputs, enc_hidden = self.encoder(src, src_lens)
            src_len = encoder_outputs.size(1)
            mask = self.create_mask(src_lens.tolist(), src_len)
            
            if isinstance(enc_hidden, tuple):
                dec_hidden = enc_hidden
            else:
                dec_hidden = enc_hidden
            
            input_token = torch.tensor([1], device=self.device)
            output_seq = []
            for _ in range(max_len):
                output, dec_hidden, _ = self.decoder(input_token, dec_hidden, encoder_outputs, mask)
                top1 = output.argmax(1)
                if top1.item() == 2:
                    break
                output_seq.append(top1.item())
                input_token = top1
        return output_seq

# =======================
# Training and Evaluation
# =======================
def train(model, loader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    total_correct = 0
    total_tokens = 0
    
    for batch in loader:
        src, trg, src_lens, _, _, _ = batch
        src, trg = src.to(device), trg.to(device)
        optimizer.zero_grad()
        output = model((src, src_lens), trg)
        
        # Calculate loss
        output_dim = output.shape[-1]
        loss = criterion(output[:, 1:].reshape(-1, output_dim), trg[:, 1:].reshape(-1))
        loss.backward()
        optimizer.step()
        
        # Calculate accuracy
        pred = output.argmax(dim=2)
        mask = (trg[:, 1:] != 0)
        correct = ((pred[:, 1:] == trg[:, 1:]) & mask).sum().item()
        total_correct += correct
        total_tokens += mask.sum().item()
        total_loss += loss.item()
    
    avg_loss = total_loss / len(loader)
    accuracy = (total_correct / total_tokens) * 100 if total_tokens > 0 else 0
    return avg_loss, accuracy

def evaluate(model, loader, criterion, device):
    model.eval()
    total_loss = 0
    total_correct = 0
    total_tokens = 0
    
    with torch.no_grad():
        for batch in loader:
            src, trg, src_lens, _, _, _ = batch
            src, trg = src.to(device), trg.to(device)
            output = model((src, src_lens), trg, teacher_forcing_ratio=0)
            
            # Calculate loss
            output_dim = output.shape[-1]
            loss = criterion(output[:, 1:].reshape(-1, output_dim), trg[:, 1:].reshape(-1))
            
            # Calculate accuracy
            pred = output.argmax(dim=2)
            mask = (trg[:, 1:] != 0)
            correct = ((pred[:, 1:] == trg[:, 1:]) & mask).sum().item()
            total_correct += correct
            total_tokens += mask.sum().item()
            total_loss += loss.item()
    
    avg_loss = total_loss / len(loader)
    accuracy = (total_correct / total_tokens) * 100 if total_tokens > 0 else 0
    return avg_loss, accuracy

# =======================
# Main Execution
# =======================
def main():
    wandb.init(config=best_config, project="dakshina-translit")

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Initialize vocabularies
    inp_vocab = Vocab()
    out_vocab = Vocab()

    # Load datasets
    train_data = TransliterationDataset("/kaggle/input/devnagiridata/hi.translit.sampled.train.tsv", inp_vocab, out_vocab)
    dev_data = TransliterationDataset("/kaggle/input/devnagiridata/hi.translit.sampled.dev.tsv", inp_vocab, out_vocab)
    test_data = TransliterationDataset("/kaggle/input/devnagiridata/hi.translit.sampled.test.tsv", inp_vocab, out_vocab, is_test=True)

    # Create data loaders
    train_loader = DataLoader(train_data, batch_size=best_config['batch_size'], 
                            shuffle=True, collate_fn=collate_fn)
    dev_loader = DataLoader(dev_data, batch_size=best_config['batch_size'],
                          shuffle=False, collate_fn=collate_fn)
    test_loader = DataLoader(test_data, batch_size=1, shuffle=False, collate_fn=collate_fn)

    # Initialize model
    encoder = Encoder(inp_vocab.size, best_config['embedding_dim'], 
                     best_config['hidden_dim'], best_config['enc_layers'], 
                     best_config['cell_type'], best_config['dropout'])
    
    decoder = Decoder(out_vocab.size, best_config['embedding_dim'],
                     best_config['hidden_dim'], best_config['dec_layers'],
                     best_config['cell_type'], best_config['dropout'],
                     best_config['attention_type'])
    
    model = Seq2Seq(encoder, decoder, device).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=best_config['learning_rate'])
    criterion = nn.CrossEntropyLoss(ignore_index=0)

    # Training loop
    best_val_loss = float('inf')
    for epoch in range(best_config['epochs']):
        train_loss, train_acc = train(model, train_loader, criterion, optimizer, device)
        val_loss, val_acc = evaluate(model, dev_loader, criterion, device)
        
        print(f"\nEpoch {epoch+1}/{best_config['epochs']}")
        print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%")
        print(f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}%")

        wandb.log({
            "epoch": epoch + 1,
            "train_loss": train_loss,
            "train_acc": train_acc,
            "val_loss": val_loss,
            "val_acc": val_acc
        })
        
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), "best_model.pth")
            print("Best model saved!")

    # Test evaluation
    model.load_state_dict(torch.load("best_model.pth"))
    model.eval()
    
    total_correct = 0
    total_samples = 0
    predictions = []
    
    with torch.no_grad():
        for batch in test_loader:
            src, lat, dev, src_lens = batch
            src = src.to(device)
            pred_ids = model.predict(src, src_lens)
            pred_str = out_vocab.decode(pred_ids)
            true_str = dev[0]
            
            predictions.append({
                'input': lat[0],
                'true': true_str,
                'pred': pred_str
            })
            
            if pred_str == true_str:
                total_correct += 1
            total_samples += 1

    # Calculate accuracy
    accuracy = 100 * total_correct / total_samples
    print(f"\nTest Accuracy: {accuracy:.2f}%")
    wandb.log({"test_acc": accuracy})

     # Create and log a table of predictions
    table = wandb.Table(columns=["Input", "True", "Predicted"])
    for p in predictions[:20]:  # Log first 20 predictions
        table.add_data(p['input'], p['true'], p['pred'])
    
    wandb.log({
        "predictions": table,
        "test_accuracy": accuracy
    })

    
    # Save predictions
    with open("test_predictions.txt", "w", encoding="utf-8") as f:
        f.write(f"Test Accuracy: {accuracy:.2f}%\n\n")
        for p in predictions[:20]:
            f.write(f"Input: {p['input']}\n")
            f.write(f"True: {p['true']}\n")
            f.write(f"Pred: {p['pred']}\n\n")
    
    # Print random samples
    print("\nRandom Samples:")
    samples = random.sample(predictions, 5)
    for sample in samples:
        print(f"Input: {sample['input']}")
        print(f"True: {sample['true']}")
        print(f"Pred: {sample['pred']}\n")

if __name__ == "__main__":
    main()

wandb: Currently logged in as: manglesh_dlass3 (manglesh_dl_ass3) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin



Epoch 1/15
Train Loss: 1.6700 | Train Acc: 49.26%
Val Loss: 0.9560 | Val Acc: 69.54%
Best model saved!

Epoch 2/15
Train Loss: 0.8696 | Train Acc: 73.11%
Val Loss: 0.9008 | Val Acc: 71.14%
Best model saved!

Epoch 3/15
Train Loss: 0.7541 | Train Acc: 76.55%
Val Loss: 0.8106 | Val Acc: 73.87%
Best model saved!

Epoch 4/15
Train Loss: 0.6842 | Train Acc: 78.64%
Val Loss: 0.8309 | Val Acc: 74.67%

Epoch 5/15
Train Loss: 0.6603 | Train Acc: 79.25%
Val Loss: 0.8221 | Val Acc: 75.44%

Epoch 6/15
Train Loss: 0.6305 | Train Acc: 80.20%
Val Loss: 0.7608 | Val Acc: 75.27%
Best model saved!

Epoch 7/15
Train Loss: 0.6054 | Train Acc: 80.90%
Val Loss: 0.7964 | Val Acc: 75.49%

Epoch 8/15
Train Loss: 0.5982 | Train Acc: 81.02%
Val Loss: 0.8013 | Val Acc: 75.42%

Epoch 9/15
Train Loss: 0.5822 | Train Acc: 81.49%
Val Loss: 0.7886 | Val Acc: 75.25%

Epoch 10/15
Train Loss: 0.5663 | Train Acc: 81.96%
Val Loss: 0.7844 | Val Acc: 75.90%

Epoch 11/15
Train Loss: 0.5470 | Train Acc: 82.60%
Val Loss: 0.771